#### Домашняя работа №2 "Реализация наивного Байесовского классификатора"
##### Работу выполнила студентка группы Т12О-101М-20 


##### Трусова В. Л.

In [63]:
import pandas as pd
import numpy as np
import string

from typing import List
from copy import deepcopy
from collections import defaultdict

In [5]:
filename = 'data/sms_spam_collection.tar.gz'

df = pd.read_csv(
    filename,
    compression='gzip',
    header=1,
    sep='\t',
    encoding='utf8',
    names=['class', 'sms_text'],
    error_bad_lines=False
)


df.head(5)

,class,sms_text
0,spam,Free entry in 2 a wkly comp to win FA Cup fina...
1,ham,U dun say so early hor... U c already then say...
2,ham,"Nah I don't think he goes to usf, he lives aro..."
3,spam,FreeMsg Hey there darling it's been 3 week's n...
4,ham,Even my brother is not like to speak with me. ...


Проверяем, сколько у нас всего объектов в датасете

In [89]:
num_objects, num_features = df.shape
print(num_objects, num_features)

5571 3


Целевая переменная (target) в столбце `class`

In [90]:
df['class'].head()

0    spam
1     ham
2     ham
3    spam
4     ham
Name: class, dtype: object

Демонстрация того, как получить булеву маску для датафрейма

In [91]:
SPAM_CLASS = 'spam'
NOT_SPAM_CLASS = 'ham'

df['class'] == SPAM_CLASS

0        True
1       False
2       False
3        True
4       False
        ...  
5566    False
5567    False
5568    False
5569    False
5570    False
Name: class, Length: 5571, dtype: bool

Использование булевой маски для фильтрации датафрейма 

In [92]:
spam_sms_num = (df['class'] == SPAM_CLASS).sum()
notspam_sms_num = (df['class'] == NOT_SPAM_CLASS).sum()

print(f'spam sms: {spam_sms_num}, not spam sms {notspam_sms_num}')

spam sms: 747, not spam sms 4823


##### Задача

считаем вероятности классов

In [93]:
# априорная вероятность класса спам
p_spam = spam_sms_num / num_objects

# априорная вероятность класса не спам
p_notspam = notspam_sms_num / num_objects

print(f'{p_spam:.4f}, {p_notspam:.4f}')

0.1341, 0.8657


Пример обработки текстовой информации - приводим к нижнему регистру

In [94]:
test_word = 'Free'.lower()

test_word

'free'

In [95]:
#пример сообщения
sms_example = df['sms_text'].values[0]

sms_example

"Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"

Пишем полезные сниппеты для трансформации текста

In [96]:
# удаляем знаки препинания

print(string.punctuation)

sms_example = ''.join([
    char
    for char in sms_example
    if char not in string.punctuation
])

sms_example

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


'Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005 Text FA to 87121 to receive entry questionstd txt rateTCs apply 08452810075over18s'

In [97]:
# приводим слова к нижнему регистру

sms_example = ' '.join([
    word.lower()
    for word in sms_example.split(sep=' ')
])

sms_example

'free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005 text fa to 87121 to receive entry questionstd txt ratetcs apply 08452810075over18s'

Объединяем сниппеты в функцию

In [121]:
"""Преобразование текста для анализа"""

def text_preprocess(sms_text: str) -> str:
    """Преобразование текста для анализа"""
    text_no_punctuation = ''.join([
        char
        for char in sms_text
        if char not in string.punctuation
    ])
    text_lowercase = ' '.join([
        word.lower()
        for word in text_no_punctuation.split(sep=' ')
    ])
    
    return text_lowercase

In [124]:
def text_preprocess_df(sms_text: str) -> str:
    sms_text = str(sms_text)
    text_no_punctuation = ''.join([
        char
        for char in sms_text
        if char not in string.punctuation
    ])
    text_lowercase = ' '.join([
        word.lower()
        for word in text_no_punctuation.split(sep=' ')
    ])
    
    return text_lowercase

In [125]:
# проверяем работу функции на примере
sms_example = df['sms_text'].values[6]
print(text_preprocess_df(sms_example))

winner as a valued network customer you have been selected to receivea £900 prize reward to claim call 09061701461 claim code kl341 valid 12 hours only


Трансформируем каждую строчку датафрейма

In [126]:
df = df.assign(
    processed_text=df['sms_text'].apply(text_preprocess_df)
)

df.head()

,class,sms_text,processed_text
0,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...
1,ham,U dun say so early hor... U c already then say...,u dun say so early hor u c already then say
2,ham,"Nah I don't think he goes to usf, he lives aro...",nah i dont think he goes to usf he lives aroun...
3,spam,FreeMsg Hey there darling it's been 3 week's n...,freemsg hey there darling its been 3 weeks now...
4,ham,Even my brother is not like to speak with me. ...,even my brother is not like to speak with me t...


#### Задача

Находим вероятность встретить слово в каждом из классов - это наша основная "фича" в наивном байесовском классификаторе

In [111]:
# сколько раз test_word встречаемся в документах класса spam
spam_test_word_entries = df[
    df['class'] == SPAM_CLASS]['processed_text'].apply(
    lambda row: test_word in row).sum()

# вероятность встретить слово в спам смс
print(f'P(word="{test_word}"|class=spam)={spam_test_word_entries/spam_sms_num:.4f}')

P(word="free"|class=spam)=0.2664


In [112]:
# сколько раз test_word встречаемся в документах класса spam
notspam_test_word_entries = df[
    df['class'] == NOT_SPAM_CLASS]['processed_text'].apply(
    lambda row: test_word in row).sum()

# вероятность встретить слово в не-спам смс
print(f'P(word="{test_word}"|class=not_spam)={notspam_test_word_entries/notspam_sms_num:.4f}')

P(word="free"|class=not_spam)=0.0137


### Вывод

слово "free" встречается в спам смс с вероятностью $26.6\%$, а в не-спаме с вероятностью $1.3\%$ - т.е. это слово является хорошим "маркером" спама

# Реализовать классификатор

Аналогично тому, как посчитали вероятности встретить слово `free` в каждом классе (спам/не спам) 
* в функции `fit()` подсчитать такие вероятности для каждого слова
* в функции `predict()` по формуле байеса (см. лекцию) вычислять вероятность принадлежности входного текста к каждому из классов

Результат предсказания - класс, вероятность принадлежности к которому больше

In [116]:
from typing import List
from copy import deepcopy
from collections import defaultdict

import numpy as np

"""имплементация наивного байесовского классификатора"""
class NaiveBayes:
    def __init__(self):
        
        self.labels = [NOT_SPAM_CLASS, SPAM_CLASS]
        self.class_labels_proba = None  # априорная вероятность класса
        self.prior_word_proba = None  # частоты фичей (токенов)
    
    def _set_labels_prior_proba(self, data: list, target: list):
        """Вычисление априорной вероятности классов"""
        class_labels_count = dict.fromkeys(self.labels, 0)

        num_objects = len(data)
        for label in target:
            try:
                class_labels_count[label] += 1
            except KeyError:
                print(f'Некорректное значение метки: {label}')

        class_labels_proba = {
            i: j / num_objects for i, j in class_labels_count.items()
        }
        self.class_labels_proba = class_labels_proba
    
    def _tokenize_text(self, text):
        tokens = []
        try:
            processed_text = text_preprocess(text)
            tokens = processed_text.split(' ')
        except TypeError:
            print(f'Ошибка при обработке текста sms: {text}')
        return tokens
    
    def _set_word_count(self, data, target):
        def_dict_obj = defaultdict(int)
        word_count_dict_by_class = dict.fromkeys(self.labels)
        for label in word_count_dict_by_class:
            word_count_dict_by_class[label] = deepcopy(def_dict_obj)
        word_count_dict_total = deepcopy(def_dict_obj)
        for label, text in zip(target, data):
                for token in self._tokenize_text(text):
                    word_count_dict_by_class[label][token] += 1 
                    word_count_dict_total[token] += 1
        # переходим от счётчиков к частотам
        total_token_count = sum((count for token, count in word_count_dict_total.items()))
        print(f'total_token_count={total_token_count}')
        for label in word_count_dict_by_class:
            for token in word_count_dict_by_class[label]:
                word_count_dict_by_class[label][token] = word_count_dict_by_class[label][token] / total_token_count
        self.prior_word_proba = word_count_dict_by_class

    def fit(self, data: list, target: list):
        """Обучение статистик по датасету

        :param data: массив документов, каждый документ - объект типа str
        :param target: массив меток объектов
        :return:
        """
        if not isinstance(data, list):
            raise ValueError('Аргумент data должен иметь тип list')
        if not isinstance(target, list):
            raise ValueError('Аргумент target должен иметь тип list')
        print('Данные инициализированы!')
        self._set_labels_prior_proba(data, target)
        print(f'Априорные вероятности классов {self.class_labels_proba}')
        self._set_word_count(data, target)
        print('Обучили априорные вероятности слов')
        

    def _predict_proba(self, data: list) -> List[tuple]:
        """Предсказываем класс для текстовой смс

        :param data: массив документов, для каждого из которых нужно предсказать метку
        :return: вероятности для каждого из классов
        """
        prediction = []
        for obj in data:
            posterior_class_proba = defaultdict(lambda: 1)
            for token in self._tokenize_text(obj):
                for label in self.labels:
                    posterior_class_proba[label] *= self.prior_word_proba[label][token]
            # сохраняем для каждой метки класса - сколько меток, таков и размер uple
            prediction.append(
                tuple(
                    posterior_class_proba[label] for label in self.labels
                )
            )
        print(f'proba: {prediction}')
        return prediction
    
    def predict(self, data) -> List[str]:
        predict_labels = []
        for proba in self._predict_proba(data):
            predict_labels.append(self.labels[np.argmax(proba)])
        return predict_labels

naive_bayes = NaiveBayes()

In [117]:
naive_bayes.fit(
    data=df['sms_text'].values.tolist(),
    target=df['class'].tolist()
)

Данные инициализированы!
Некорректное значение метки: nan
Априорные вероятности классов {'ham': 0.8657332615329384, 'spam': 0.13408723747980614}
Ошибка при обработке текста sms: nan
total_token_count=87507
Обучили априорные вероятности слов


In [118]:
naive_bayes.prior_word_proba['ham']['thank'], naive_bayes.prior_word_proba['spam']['thank']

(0.000308546744831842, 1.1427657215994148e-05)

Предсказание метки класса

In [119]:
# рандомный объект датасета

random_obj_ind = np.random.randint(low=0, high=num_objects, size=3)
random_obj_list = df['sms_text'].values[random_obj_ind].tolist()
random_target = df['class'][random_obj_ind].tolist()

print(random_obj_list)
naive_bayes.predict(
    random_obj_list
)

['K. I will sent it again', 'I think asking for a gym is the excuse for lazy people. I jog.', 'Thats cool princess! I will cover your face in hot sticky cum :)']
proba: [(2.512141089524242e-16, 0.0), (3.5402607457971635e-41, 0.0), (2.6156566929381783e-41, 0.0)]


['ham', 'ham', 'ham']